## Open Exploration

Explanation to what we did + table of contents

### Import Libraries

In [1]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy

# Import problem formulation
from problem_formulation import get_model_for_problem_formulation

# EMA workbench imports
from ema_workbench import (Model, Policy, MultiprocessingEvaluator, ScalarOutcome, RealParameter, 
                           IntegerParameter, CategoricalParameter, optimize, Scenario, 
                           Constant, SequentialEvaluator)
#from ema_workbench.em_framework.evaluators import perform_experiments,LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator
from ema_workbench.em_framework.samplers import sample_uncertainties, sample_levers
from ema_workbench.util import ema_logging, save_results, load_results

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## Worst Case Scenario 

In [2]:
# define the problem formulation between 0 and 9
# Problem formulation 6 is specific to Dike Ring 3
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [3]:
# Set uncertainties and levers variables
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

In [4]:
# Create a reference policy where no action is taken
policies = [Policy('null_policy', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':0,
                                  'A.2_DikeIncrease 0':0,
                                  'A.3_DikeIncrease 0':0,
                                  'A.4_DikeIncrease 0':0,
                                  'A.5_DikeIncrease 0':0,
                                  'A.1_DikeIncrease 1':0,
                                  'A.2_DikeIncrease 1':0,
                                  'A.3_DikeIncrease 1':0,
                                  'A.4_DikeIncrease 1':0,
                                  'A.5_DikeIncrease 1':0,
                                  'A.1_DikeIncrease 2':0,
                                  'A.2_DikeIncrease 2':0,
                                  'A.3_DikeIncrease 2':0,
                                  'A.4_DikeIncrease 2':0,
                                  'A.5_DikeIncrease 2':0,
                                  'EWS_DaysToThreat':0})
                                  ]

In [5]:
n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [00:12<00:00,  7.87it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [6]:
#Save the results
save_results([experiments, outcomes], 'results/10000Scenarios_NullPolicy_PF6.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz


In [7]:
# load the worst case results for further analysis 
experiments, outcomes = load_results('results/10000Scenarios_NullPolicy_PF6.tar.gz')

[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz


In [8]:
outcomes

{'A3 Expected Annual Damage': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'A3 Aggr Expected Number of Deaths': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'A3 Dike Investment Costs': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


## Here actual Open exploration shit


### show that random policy is better than no policy

In [9]:
# Create a policy where action is taken randomly
# compare the performance of that policy to the previously computed null_policy


import random # for randomly deciding to switch policy on/ off

policies = [Policy('null_policy', **{'3_RfR 0':0,
                                  '3_RfR 1':0,
                                  '3_RfR 2':0,
                                  'A.1_DikeIncrease 0':0,
                                  'A.2_DikeIncrease 0':0,
                                  'A.3_DikeIncrease 0':0,
                                  'A.4_DikeIncrease 0':0,
                                  'A.5_DikeIncrease 0':0,
                                  'A.1_DikeIncrease 1':0,
                                  'A.2_DikeIncrease 1':0,
                                  'A.3_DikeIncrease 1':0,
                                  'A.4_DikeIncrease 1':0,
                                  'A.5_DikeIncrease 1':0,
                                  'A.1_DikeIncrease 2':0,
                                  'A.2_DikeIncrease 2':0,
                                  'A.3_DikeIncrease 2':0,
                                  'A.4_DikeIncrease 2':0,
                                  'A.5_DikeIncrease 2':0,
                                  'EWS_DaysToThreat':0})
                ('alpha_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)})
                ('beta_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)})
                ('gamma_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)})                  
                                  ]

TypeError: 'Policy' object is not callable

In [ ]:
n_scenarios = 5000

ema_logging.log_to_stderr(ema_logging.INFO)

# Model Imports
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation

dike_model, time_step = get_model_for_problem_formulation(7)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies)

In [ ]:
#Save the results
save_results([experiments, outcomes], 'results/10000Scenarios_RandomPolicies_PF7.tar.gz')

0